# Training a Neural Network with PyTorch

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd

## Load data

In [ ]:
data = pd.read_csv('/content/sample_data/california_housing_train.csv')
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [ ]:
#Fill NaN values
data = data.fillna(0)
#Normalize values
data = (data-data.mean())/data.std()
#Separate features and targets
x_df = pd.DataFrame(data, columns=data.columns[:-1])
y_df = pd.DataFrame(data, columns=[data.columns[-1]])
#Save in tensors
x = torch.tensor(x_df.values, dtype=torch.float)
y = torch.tensor(y_df.values, dtype=torch.float)

print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

x shape: torch.Size([17000, 8])
y shape: torch.Size([17000, 1])


## Create a Neural Network

In [ ]:
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
        super(Net, self).__init__()
        
        self.linear1 = nn.Linear(D_in, H1)
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, H3)
        self.linear4 = nn.Linear(H3, D_out)
        
    def forward(self, x):
        h1 = self.linear1(x)
        h2 = self.linear2(h1)
        h3 = self.linear3(h2)
        out = self.linear4(h3)
        return out

In [ ]:
#Define layer sizes
D_in = x.shape[1]
H1 = 128
H2 = 64
H3 = 32
D_out = 1

#Define Hyperparameters
learning_rate = 1e-4

#Initialize model, loss, optimizer
model = Net(D_in, H1, H2, H3, D_out)
loss_func = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#Initialize dataloader
dataset = torch.utils.data.TensorDataset(x, y)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
model

Net(
  (linear1): Linear(in_features=8, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=32, bias=True)
  (linear4): Linear(in_features=32, out_features=1, bias=True)
)

## Train Network

In [ ]:
for epoch in range(100):
  batch_losses = []

  for x_batch, y_batch in dataloader:
    y_pred = model(x_batch)
    
    loss = loss_func(y_pred, y_batch)
    batch_losses.append(loss.item())
    
    #Delete previously stored gradients
    optimizer.zero_grad()
    #Perform backpropagation starting from the loss calculated in this epoch
    loss.backward()
    #Update model's weights based on the gradients calculated during backprop
    optimizer.step()
  
  print(f"Epoch {epoch:3}: Loss = {sum(batch_losses)/len(dataloader):.5f}")
    

Epoch   0: Loss = 23.03230
Epoch   1: Loss = 23.00585
Epoch   2: Loss = 23.00443
Epoch   3: Loss = 23.02430
Epoch   4: Loss = 23.02099
Epoch   5: Loss = 23.03321
Epoch   6: Loss = 23.00538
Epoch   7: Loss = 23.00858
Epoch   8: Loss = 23.02187
Epoch   9: Loss = 23.00640
Epoch  10: Loss = 23.03558
Epoch  11: Loss = 23.00467
Epoch  12: Loss = 23.02168
Epoch  13: Loss = 23.03198
Epoch  14: Loss = 23.02035
Epoch  15: Loss = 23.03098
Epoch  16: Loss = 23.02864
Epoch  17: Loss = 23.00796
Epoch  18: Loss = 22.98849
Epoch  19: Loss = 23.01406
Epoch  20: Loss = 23.02653
Epoch  21: Loss = 22.98048
Epoch  22: Loss = 23.05250
Epoch  23: Loss = 23.02554
Epoch  24: Loss = 22.98849
Epoch  25: Loss = 23.02380
Epoch  26: Loss = 23.03623
Epoch  27: Loss = 23.03491
Epoch  28: Loss = 22.98315
Epoch  29: Loss = 23.01919
Epoch  30: Loss = 23.01855
Epoch  31: Loss = 23.03876
Epoch  32: Loss = 23.01102
Epoch  33: Loss = 23.01832
Epoch  34: Loss = 23.04427
Epoch  35: Loss = 23.01305
Epoch  36: Loss = 23.03183
E